In [1]:
%run configuration.ipynb

In [2]:
%run churn_model.ipynb

In [3]:
%run train_validate_test_split.ipynb

In [8]:
import pickle

# load the training and test set

In [5]:
npy_data = {}

for c in profile_attributes+even_attributes:
    npy_data[c] = np.load(f'{npy_transformed_folder}\\{c}.npy', allow_pickle=True)

X = []
for c in feature_attributes:
    x = npy_data[c]
    X.append(x)

y = npy_data[label_column]

In [6]:
X_training = [x[x_idx_train] for x in X]
y_training = y[x_idx_train]

X_test = [x[x_idx_test] for x in X]
y_test = y[x_idx_test]

# load the best parameters

In [9]:
f = open("best_parameters.pkl", "rb")
best_parameters = pickle.load(f)
print(best_parameters)
f.close()

{'units': 128, 'epochs': 3, 'emb_dim': 128, 'dropout_rate': 0.3}


# build the model

In [13]:
churn = churn_model(
    emb_dim = best_parameters['emb_dim'],
    dropout_rate = best_parameters['dropout_rate'],
    units = best_parameters['units'],
)
churn.build_model()

# train the model

In [14]:
print(f'training the model over the trainig set with parameters {best_parameters}')

churn.model.fit(
    x=X_training,
    y=y_training,
    batch_size=batch_size,
    epochs=best_parameters['epochs'],
    verbose="auto",
    )

Epoch 1/3
8964/8964 [==============================] - 75s 8ms/step - loss: 0.5072 - auc: 0.8343 - acc: 0.7716
Epoch 2/3
8964/8964 [==============================] - 69s 8ms/step - loss: 0.4767 - auc: 0.8532 - acc: 0.7879
Epoch 3/3
8964/8964 [==============================] - 67s 7ms/step - loss: 0.4710 - auc: 0.8567 - acc: 0.7917


# evaluate the model

In [15]:
print(f'evaluating the model over the test set')

loss = churn.model.evaluate(
    x=X_test,
    y=y_test,
    batch_size=batch_size,
    )

2988/2988 [==============================] - 12s 4ms/step - loss: 0.4543 - auc: 0.8682 - acc: 0.7979


In [17]:
print(f'evaluatoin result: auc: {loss[1]:0.4f}, acc: {loss[2]:0.4f}')

auc: 0.8682, acc: 0.7979


# end